In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations,permutations
from datetime import datetime, timedelta
import random

In [ ]:
hotel_rates_df = pd.read_csv("/content/hotel_rates.csv")
hotel_rates_df.head()

,City,Hotel,Rate (INR)
0,Delhi,Taj,9000
1,Delhi,Marriott,8000
2,Delhi,ITC,7500
3,Mumbai,Taj,10000
4,Mumbai,Marriott,9000


In [ ]:
flight_costs_df = pd.read_csv("/content/flight_costs.csv")
flight_costs_df.head()

,Route,Air India (INR),Vistara (INR),IndiGo (INR)
0,Delhi-Mumbai,5000,5500,4800
1,Delhi-Hyderabad,4500,5000,4300
2,Delhi-Chennai,4500,5000,4300
3,Delhi-Kolkata,5000,5500,4800
4,Delhi-Mohali,4500,5000,4300


In [ ]:
flight_costs_df.head()

,Route,Air India (INR),Vistara (INR),IndiGo (INR)
0,Delhi-Mumbai,5000,5500,4800
1,Delhi-Hyderabad,4500,5000,4300
2,Delhi-Chennai,4500,5000,4300
3,Delhi-Kolkata,5000,5500,4800
4,Delhi-Mohali,4500,5000,4300


In [ ]:
# Define the teams and their corresponding venues
teams = [
    "Delhi Capitals", "Kolkata Knight Riders", "Mumbai Indians",
    "Rajasthan Royals", "Royal Challengers Bengaluru", "Sunrisers Hyderabad",
    "Chennai Super Kings", "Punjab Kings"
]
venues = {
    "Delhi Capitals": "Arun Jaitley Stadium, Delhi",
    "Kolkata Knight Riders": "Eden Gardens, Kolkata",
    "Mumbai Indians": "Wankhede Stadium, Mumbai",
    "Rajasthan Royals": "Anil Agarwal Stadium, Jaipur",
    "Royal Challengers Bengaluru": "M. Chinnaswamy Stadium, Bangalore",
    "Sunrisers Hyderabad": "Rajiv Gandhi Stadium, Hyderabad",
    "Chennai Super Kings": "M.A. Chidambaram Stadium, Chennai",
    "Punjab Kings": "Cricket Association's Bindra Stadium, Mohali"
}

# Generate all possible matches (home and away)
match_list = []
for team1, team2 in combinations(teams, 2):
    match_list.append((team1, team2, venues[team1]))  # team1 at home
    match_list.append((team2, team1, venues[team2]))  # team2 at home

# Define the schedule starting date
start_date = pd.Timestamp("2024-05-24")

# Create the match schedule
schedule = []
current_date = start_date
for i, match in enumerate(match_list):
    # Decide match times based on the day of the week
    if current_date.weekday() in [4, 5, 6]:  # Friday, Saturday, Sunday
        times = ["2:30 PM", "6:30 PM"] if current_date.weekday() != 4 else ["6:30 PM"]
    else:
        times = ["6:30 PM"]

    for time in times:
        schedule.append((current_date.strftime("%d-%m-%Y"), current_date.strftime("%A"), time, f"{match[0]} vs {match[1]}", match[2]))
        i += 1
        if i >= len(match_list):
            break
    current_date += pd.DateOffset(days=1 if current_date.weekday() in [3, 4, 5] else 2)

# Convert the list to a DataFrame
full_schedule_df = pd.DataFrame(schedule, columns=["Date", "Day", "Time", "Match", "Venue"])


In [ ]:
full_schedule_df.head()

,Date,Day,Time,Match,Venue
0,24-05-2024,Friday,6:30 PM,Delhi Capitals vs Kolkata Knight Riders,"Arun Jaitley Stadium, Delhi"
1,25-05-2024,Saturday,2:30 PM,Kolkata Knight Riders vs Delhi Capitals,"Eden Gardens, Kolkata"
2,25-05-2024,Saturday,6:30 PM,Kolkata Knight Riders vs Delhi Capitals,"Eden Gardens, Kolkata"
3,26-05-2024,Sunday,2:30 PM,Delhi Capitals vs Mumbai Indians,"Arun Jaitley Stadium, Delhi"
4,26-05-2024,Sunday,6:30 PM,Delhi Capitals vs Mumbai Indians,"Arun Jaitley Stadium, Delhi"


In [ ]:
# Define teams and their home cities
teams = {
    "Delhi Capitals": "Delhi",
    "Kolkata Knight Riders": "Kolkata",
    "Mumbai Indians": "Mumbai",
    "Rajasthan Royals": "Jaipur",
    "Royal Challengers Bengaluru": "Bangalore",
    "Sunrisers Hyderabad": "Hyderabad",
    "Chennai Super Kings": "Chennai",
    "Punjab Kings": "Mohali"
}

# Generate all match permutations
matches = list(permutations(teams.keys(), 2))

# Schedule setup
start_date = datetime(2024, 5, 24)
match_dates = []
day_increment = 0

for home_team, away_team in matches:
    home_city = teams[home_team]
    away_city = teams[away_team]
    match_date = start_date + timedelta(days=day_increment)
    match_dates.append({
        'Date': match_date.strftime('%d-%m-%Y'),
        'Home Team': home_team,
        'Away Team': away_team,
        'Venue': f"{home_city} Stadium"
    })

    # Select the cheapest flight for the away team
    route = f"{away_city}-{home_city}"
    flight_costs = flight_costs_df.loc[flight_costs_df['Route'] == route]
    min_flight_cost = flight_costs.iloc[:, 1:].min(axis=1).values[0]  # Assuming costs start from second column

    # Get hotel cost for the away team
    hotel_cost = hotel_rates_df.loc[(hotel_rates_df['City'] == home_city) & (hotel_rates_df['Hotel'] == 'Taj'), 'Rate (INR)'].values[0]

    # Update the date for the next match
    day_increment += 3  # Assumes a match every three days

# Convert match details into a DataFrame
schedule_df = pd.DataFrame(match_dates)

print(schedule_df.head())

         Date       Home Team                    Away Team          Venue
0  24-05-2024  Delhi Capitals        Kolkata Knight Riders  Delhi Stadium
1  27-05-2024  Delhi Capitals               Mumbai Indians  Delhi Stadium
2  30-05-2024  Delhi Capitals             Rajasthan Royals  Delhi Stadium
3  02-06-2024  Delhi Capitals  Royal Challengers Bengaluru  Delhi Stadium
4  05-06-2024  Delhi Capitals          Sunrisers Hyderabad  Delhi Stadium


In [ ]:
# Define teams and their home cities
teams = {
    "Delhi Capitals": "Delhi",
    "Kolkata Knight Riders": "Kolkata",
    "Mumbai Indians": "Mumbai",
    "Rajasthan Royals": "Jaipur",
    "Royal Challengers Bengaluru": "Bangalore",
    "Sunrisers Hyderabad": "Hyderabad",
    "Chennai Super Kings": "Chennai",
    "Punjab Kings": "Mohali"
}
# Generate all match permutations
matches = list(permutations(teams.keys(), 2))

# Randomly shuffle matches
random.shuffle(matches)

# Set the start date for the schedule
start_date = datetime(2024, 5, 24)

# Assign dates to matches with at least two days break between each match for a team
schedule = []
team_next_available_date = {team: start_date for team in teams}

for home_team, away_team in matches:
    home_city = teams[home_team]
    away_city = teams[away_team]

    # Find the earliest possible date for this match considering both teams' schedules
    earliest_date = max(team_next_available_date[home_team], team_next_available_date[away_team]) + timedelta(days=2)
    schedule.append({
        'Date': earliest_date.strftime('%Y-%m-%d'),
        'Home Team': home_team,
        'Away Team': away_team,
        'Venue': f"{home_city} Stadium",
        'Flight Cost (INR)': flight_costs_df.loc[flight_costs_df['Route'] == f"{away_city}-{home_city}", 'IndiGo (INR)'].values[0],
        'Hotel Cost (INR)': hotel_rates_df.loc[hotel_rates_df['City'] == home_city, 'Rate (INR)'].values[0]
    })

    # Update the next available date for both teams
    team_next_available_date[home_team] = earliest_date + timedelta(days=1)
    team_next_available_date[away_team] = earliest_date + timedelta(days=1)

# Convert the schedule into a DataFrame
schedule_df = pd.DataFrame(schedule)

# Print the schedule
print(schedule_df.head())


         Date                    Home Team            Away Team  \
0  2024-05-26  Royal Challengers Bengaluru  Chennai Super Kings   
1  2024-05-26               Mumbai Indians       Delhi Capitals   
2  2024-05-29          Sunrisers Hyderabad  Chennai Super Kings   
3  2024-06-01                 Punjab Kings  Chennai Super Kings   
4  2024-06-01             Rajasthan Royals  Sunrisers Hyderabad   

               Venue  Flight Cost (INR)  Hotel Cost (INR)  
0  Bangalore Stadium               4300              9500  
1     Mumbai Stadium               4800             10000  
2  Hyderabad Stadium               3800              7500  
3     Mohali Stadium               3800              6000  
4     Jaipur Stadium               3800              8000  


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import permutations
import random

# Define teams and their home cities
teams = {
    "Delhi Capitals": "Delhi",
    "Kolkata Knight Riders": "Kolkata",
    "Mumbai Indians": "Mumbai",
    "Rajasthan Royals": "Jaipur",
    "Royal Challengers Bengaluru": "Bangalore",
    "Sunrisers Hyderabad": "Hyderabad",
    "Chennai Super Kings": "Chennai",
    "Punjab Kings": "Mohali"
}



# Generate all match permutations
matches = list(permutations(teams.keys(), 2))

# Randomly shuffle matches
random.shuffle(matches)

# Set the start date for the schedule
start_date = datetime(2024, 5, 24)

# Assign dates to matches with at least two days break between each match for a team
schedule = []
team_next_available_date = {team: start_date for team in teams}

for home_team, away_team in matches:
    home_city = teams[home_team]
    away_city = teams[away_team]

    # Find the earliest possible date for this match considering both teams' schedules
    earliest_date = max(team_next_available_date[home_team], team_next_available_date[away_team]) + timedelta(days=2)

    # Determine if it's a weekend
    weekend_multiplier = 1.2 if earliest_date.weekday() in [5, 6] else 1.0
    hotel_weekend_multiplier = 1.1 if earliest_date.weekday() in [5, 6] else 1.0

    # Select the cheapest flight and adjust if weekend
    flight_cost = min(flight_costs_df.loc[flight_costs_df['Route'] == f"{away_city}-{home_city}", ['Air India (INR)', 'Vistara (INR)', 'IndiGo (INR)']].values[0])
    adjusted_flight_cost = flight_cost * weekend_multiplier

    # Get hotel cost for the away team and adjust if weekend
    hotel_cost = hotel_rates_df.loc[hotel_rates_df['City'] == home_city, 'Rate (INR)'].values[0]
    adjusted_hotel_cost = hotel_cost * hotel_weekend_multiplier

    # Add to schedule
    schedule.append({
        'Date': earliest_date.strftime('%Y-%m-%d'),
        'Home Team': home_team,
        'Away Team': away_team,
        'Venue': f"{home_city} Stadium",
        'Flight Cost (INR)': adjusted_flight_cost,
        'Hotel Cost (INR)': adjusted_hotel_cost
    })

    # Update the next available date for both teams
    team_next_available_date[home_team] = earliest_date + timedelta(days=1)
    team_next_available_date[away_team] = earliest_date + timedelta(days=1)

# Convert the schedule into a DataFrame
schedule_df = pd.DataFrame(schedule)

# Print the schedule
print(schedule_df.head())


         Date            Home Team                    Away Team  \
0  2024-05-26  Sunrisers Hyderabad          Chennai Super Kings   
1  2024-05-26         Punjab Kings             Rajasthan Royals   
2  2024-05-26       Mumbai Indians  Royal Challengers Bengaluru   
3  2024-05-29     Rajasthan Royals        Kolkata Knight Riders   
4  2024-05-29       Mumbai Indians               Delhi Capitals   

               Venue  Flight Cost (INR)  Hotel Cost (INR)  
0  Hyderabad Stadium             4560.0            8250.0  
1     Mohali Stadium             4560.0            6600.0  
2     Mumbai Stadium             5760.0           11000.0  
3     Jaipur Stadium             4300.0            8000.0  
4     Mumbai Stadium             4800.0           10000.0  


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import permutations
import random

# Define teams and their home cities
teams = {
    "Delhi Capitals": "Delhi",
    "Kolkata Knight Riders": "Kolkata",
    "Mumbai Indians": "Mumbai",
    "Rajasthan Royals": "Jaipur",
    "Royal Challengers Bengaluru": "Bangalore",
    "Sunrisers Hyderabad": "Hyderabad",
    "Chennai Super Kings": "Chennai",
    "Punjab Kings": "Mohali"
}

# Assume flight_costs_df and hotel_rates_df are defined elsewhere

# Generate all match permutations
matches = list(permutations(teams.keys(), 2))

# Randomly shuffle matches
random.shuffle(matches)

# Set the start date for the schedule
start_date = datetime(2024, 5, 24)
end_date = datetime(2024, 7, 20)  # Matches must end by this date

# Assign dates to matches with at least two days break between each match for a team
schedule = []
team_next_available_date = {team: start_date for team in teams}

for home_team, away_team in matches:
    home_city = teams[home_team]
    away_city = teams[away_team]

    # Find the earliest possible date for this match considering both teams' schedules
    earliest_date = max(team_next_available_date[home_team], team_next_available_date[away_team]) + timedelta(days=2)

    # Check if the match date is beyond the end date
    if earliest_date > end_date:
        continue  # Skip scheduling this match if it falls outside the permitted dates

    # Determine if it's a weekend
    weekend_multiplier = 1.2 if earliest_date.weekday() in [5, 6] else 1.0
    hotel_weekend_multiplier = 1.1 if earliest_date.weekday() in [5, 6] else 1.0

    # Select the cheapest flight and adjust if weekend
    flight_cost = min(flight_costs_df.loc[flight_costs_df['Route'] == f"{away_city}-{home_city}", ['Air India (INR)', 'Vistara (INR)', 'IndiGo (INR)']].values[0])
    adjusted_flight_cost = flight_cost * weekend_multiplier

    # Get hotel cost for the away team and adjust if weekend
    hotel_cost = hotel_rates_df.loc[hotel_rates_df['City'] == home_city, 'Rate (INR)'].values[0]
    adjusted_hotel_cost = hotel_cost * hotel_weekend_multiplier

    # Add to schedule
    schedule.append({
        'Date': earliest_date.strftime('%Y-%m-%d'),
        'Home Team': home_team,
        'Away Team': away_team,
        'Venue': f"{home_city} Stadium",
        'Flight Cost (INR)': adjusted_flight_cost,
        'Hotel Cost (INR)': adjusted_hotel_cost
    })

    # Update the next available date for both teams
    team_next_available_date[home_team] = earliest_date + timedelta(days=1)
    team_next_available_date[away_team] = earliest_date + timedelta(days=1)

# Convert the schedule into a DataFrame
schedule_df = pd.DataFrame(schedule)

# Print the schedule
print(schedule_df.head())


         Date            Home Team            Away Team            Venue  \
0  2024-05-26  Chennai Super Kings  Sunrisers Hyderabad  Chennai Stadium   
1  2024-05-29         Punjab Kings  Chennai Super Kings   Mohali Stadium   
2  2024-06-01  Chennai Super Kings     Rajasthan Royals  Chennai Stadium   
3  2024-06-04     Rajasthan Royals       Mumbai Indians   Jaipur Stadium   
4  2024-06-07  Chennai Super Kings       Mumbai Indians  Chennai Stadium   

   Flight Cost (INR)  Hotel Cost (INR)  
0             4560.0            7700.0  
1             3800.0            6000.0  
2             4560.0            7700.0  
3             4300.0            8000.0  
4             4300.0            7000.0  


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import permutations
import random

# Define teams and their home cities
teams = {
    "Delhi Capitals": "Delhi",
    "Kolkata Knight Riders": "Kolkata",
    "Mumbai Indians": "Mumbai",
    "Rajasthan Royals": "Jaipur",
    "Royal Challengers Bengaluru": "Bangalore",
    "Sunrisers Hyderabad": "Hyderabad",
    "Chennai Super Kings": "Chennai",
    "Punjab Kings": "Mohali"
}

# Generate all match permutations
matches = list(permutations(teams.keys(), 2))

# Randomly shuffle matches to randomize the schedule
random.shuffle(matches)

# Set the start and end dates for the schedule
start_date = datetime(2024, 5, 24)
end_date = datetime(2024, 7, 20)

# Initialize scheduling variables
schedule = []
current_date = start_date
match_index = 0

# Create the match schedule
while current_date <= end_date and match_index < len(matches):
    home_team, away_team = matches[match_index]
    venue = f"{teams[home_team]} Stadium"

    # Determine match times based on the day of the week
    if current_date.weekday() in [4, 5, 6]:  # Friday, Saturday, Sunday
        times = ["2:30 PM", "6:30 PM"] if current_date.weekday() != 4 else ["6:30 PM"]
    else:
        times = ["6:30 PM"]

    for time in times:
        schedule.append((current_date.strftime("%Y-%m-%d"), current_date.strftime("%A"), time, f"{home_team} vs {away_team}", venue))
        match_index += 1
        if match_index >= len(matches):
            break

    # Increment the date for the next match
    current_date += pd.DateOffset(days=1 if current_date.weekday() in [3, 4, 5] else 2)

# Convert the list to a DataFrame for better visualization
full_schedule_df = pd.DataFrame(schedule, columns=["Date", "Day", "Time", "Match", "Venue"])

# Print the final schedule DataFrame
print(full_schedule_df.head())


         Date       Day     Time  \
0  2024-05-24    Friday  6:30 PM   
1  2024-05-25  Saturday  2:30 PM   
2  2024-05-25  Saturday  6:30 PM   
3  2024-05-26    Sunday  2:30 PM   
4  2024-05-26    Sunday  6:30 PM   

                                               Match            Venue  
0            Kolkata Knight Riders vs Mumbai Indians  Kolkata Stadium  
1                     Delhi Capitals vs Punjab Kings    Delhi Stadium  
2                     Delhi Capitals vs Punjab Kings    Delhi Stadium  
3  Chennai Super Kings vs Royal Challengers Benga...  Chennai Stadium  
4  Chennai Super Kings vs Royal Challengers Benga...  Chennai Stadium  


In [ ]:
# Define teams and their home cities
teams = {
    "Delhi Capitals": "Delhi",
    "Kolkata Knight Riders": "Kolkata",
    "Mumbai Indians": "Mumbai",
    "Rajasthan Royals": "Jaipur",
    "Royal Challengers Bengaluru": "Bangalore",
    "Sunrisers Hyderabad": "Hyderabad",
    "Chennai Super Kings": "Chennai",
    "Punjab Kings": "Mohali"
}

# Assume these data frames are defined with realistic cost data
# flight_costs_df and hotel_rates_df

# Generate all match permutations
matches = list(permutations(teams.keys(), 2))

# Randomly shuffle matches to randomize the schedule
random.shuffle(matches)

# Set the start and end dates for the schedule
start_date = datetime(2024, 5, 24)
end_date = datetime(2024, 7, 20)

# Initialize scheduling variables
schedule = []
current_date = start_date
match_index = 0

# Create the match schedule
while current_date <= end_date and match_index < len(matches):
    home_team, away_team = matches[match_index]
    home_city = teams[home_team]
    away_city = teams[away_team]
    venue = f"{home_city} Stadium"
    route = f"{away_city}-{home_city}"

    # Determine if it's a weekend
    is_weekend = current_date.weekday() in [5, 6]
    flight_multiplier = 1.2 if is_weekend else 1
    hotel_multiplier = 1.1 if is_weekend else 1

    # Calculate costs
    flight_cost = flight_costs_df.loc[flight_costs_df['Route'] == route, 'IndiGo (INR)'].iloc[0] * flight_multiplier
    hotel_cost = hotel_rates_df.loc[(hotel_rates_df['City'] == home_city) & (hotel_rates_df['Hotel'] == 'Taj'), 'Rate (INR)'].iloc[0] * hotel_multiplier

    # Determine match times based on the day of the week
    if current_date.weekday() in [4, 5, 6]:  # Friday, Saturday, Sunday
        times = ["2:30 PM", "6:30 PM"] if current_date.weekday() != 4 else ["6:30 PM"]
    else:
        times = ["6:30 PM"]

    for time in times:
        schedule.append({
            "Date": current_date.strftime("%Y-%m-%d"),
            "Day": current_date.strftime("%A"),
            "Time": time,
            "Match": f"{home_team} vs {away_team}",
            "Venue": venue,
            "Flight Cost (INR)": flight_cost,
            "Hotel Cost (INR)": hotel_cost
        })
        match_index += 1
        if match_index >= len(matches):
            break

    # Increment the date for the next match
    current_date += pd.DateOffset(days=1 if current_date.weekday() in [3, 4, 5] else 2)

# Convert the list to a DataFrame for better visualization
full_schedule_df = pd.DataFrame(schedule)

# Print the final schedule DataFrame
print(full_schedule_df.head())


         Date       Day     Time                                        Match  \
0  2024-05-24    Friday  6:30 PM      Delhi Capitals vs Kolkata Knight Riders   
1  2024-05-25  Saturday  2:30 PM  Royal Challengers Bengaluru vs Punjab Kings   
2  2024-05-25  Saturday  6:30 PM  Royal Challengers Bengaluru vs Punjab Kings   
3  2024-05-26    Sunday  2:30 PM        Chennai Super Kings vs Delhi Capitals   
4  2024-05-26    Sunday  6:30 PM        Chennai Super Kings vs Delhi Capitals   

               Venue  Flight Cost (INR)  Hotel Cost (INR)  
0      Delhi Stadium             4800.0            9000.0  
1  Bangalore Stadium             5160.0           10450.0  
2  Bangalore Stadium             5160.0           10450.0  
3    Chennai Stadium             5160.0            7700.0  
4    Chennai Stadium             5160.0            7700.0  


In [ ]:
full_schedule_df

,Date,Day,Time,Match,Venue,Flight Cost (INR),Hotel Cost (INR)
0,2024-05-24,Friday,6:30 PM,Delhi Capitals vs Kolkata Knight Riders,Delhi Stadium,4800.0,9000.0
1,2024-05-25,Saturday,2:30 PM,Royal Challengers Bengaluru vs Punjab Kings,Bangalore Stadium,5160.0,10450.0
2,2024-05-25,Saturday,6:30 PM,Royal Challengers Bengaluru vs Punjab Kings,Bangalore Stadium,5160.0,10450.0
3,2024-05-26,Sunday,2:30 PM,Chennai Super Kings vs Delhi Capitals,Chennai Stadium,5160.0,7700.0
4,2024-05-26,Sunday,6:30 PM,Chennai Super Kings vs Delhi Capitals,Chennai Stadium,5160.0,7700.0
5,2024-05-28,Tuesday,6:30 PM,Kolkata Knight Riders vs Mumbai Indians,Kolkata Stadium,4800.0,8500.0
6,2024-05-30,Thursday,6:30 PM,Delhi Capitals vs Mumbai Indians,Delhi Stadium,4800.0,9000.0
7,2024-05-31,Friday,6:30 PM,Royal Challengers Bengaluru vs Rajasthan Royals,Bangalore Stadium,4300.0,9500.0
8,2024-06-01,Saturday,2:30 PM,Delhi Capitals vs Sunrisers Hyderabad,Delhi Stadium,5160.0,9900.0
9,2024-06-01,Saturday,6:30 PM,Delhi Capitals vs Sunrisers Hyderabad,Delhi Stadium,5160.0,9900.0
